In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [2]:
abundance = pd.read_csv("Kraken-TCGA-Voom-SNM-All-Putative-Contaminants-Removed-Data.csv", index_col=0) # Load raw data
abundance = abundance[abundance.sum(axis = 1) > 0] # remove columns that all values are 0
print(abundance.shape)

(17625, 1411)


In [3]:
# Impute missing values
feature_names = abundance.columns
imputer = KNNImputer(n_neighbors=5, weights="uniform")
abundance_imputed = imputer.fit_transform(abundance.T)
abundance_imputed = pd.DataFrame(abundance_imputed.T, columns = feature_names)
abundance_imputed.iloc[0] = abundance.iloc[0]
abundance_imputed

,k__Viruses.f__Phycodnaviridae.g__Prasinovirus,k__Viruses.o__Caudovirales.f__Siphoviridae.g__Sfi1unalikevirus,k__Viruses.o__Herpesvirales.f__Herpesviridae.g__Simplexvirus,k__Viruses.f__Poxviridae.g__Parapoxvirus,k__Viruses.f__Bicaudaviridae.g__Bicaudavirus,k__Viruses.o__Picornavirales.f__Picornaviridae.g__Aquamavirus,k__Viruses.o__Herpesvirales.f__Herpesviridae.g__Mardivirus,k__Viruses.f__Baculoviridae.g__Deltabaculovirus,k__Viruses.f__Papillomaviridae.g__Taupapillomavirus,k__Viruses.o__Caudovirales.f__Myoviridae.g__I3likevirus,...,k__Bacteria.p__Firmicutes.c__Clostridia.o__Clostridiales.f__Clostridiales_Family_XIII._Incertae_Sedis.g__Casaltella,k__Bacteria.p__Proteobacteria.c__Betaproteobacteria.o__Neisseriales.f__Chromobacteriaceae.g__Deefgea,k__Bacteria.p__Proteobacteria.c__Alphaproteobacteria.o__Rhizobiales.f__Phyllobacteriaceae.g__Aquamicrobium,k__Bacteria.p__Deferribacteres.c__Deferribacteres.o__Deferribacterales.f__Deferribacteraceae.g__Geovibrio,k__Bacteria.p__Firmicutes.c__Clostridia.o__Clostridiales.f__Ruminococcaceae.g__Subdoligranulum,contaminant1Harvard,contaminant2HarvardCanadaBaylorWashU,contaminant3AllSeqCenters,contaminant4RandomSpikesHarvard,contaminant5RandomSpikes1000
0,-0.944325,1.378696,4.257198,2.263261,0.478455,3.283590,6.175448,0.554111,1.707182,-2.701144,...,2.952431,1.053012,1.641701,1.597915,1.615110,1.613142,12.579647,12.579647,1.579534,2.482244
1,2.799427,2.095355,5.282909,0.625737,1.262373,2.598059,5.032088,-0.978710,-0.066234,0.042982,...,3.623247,2.048703,2.357091,2.339657,2.378702,2.338944,13.305450,13.305450,2.332050,2.855626
2,-0.350754,1.969313,4.789957,0.645980,1.072432,2.454999,6.929492,-1.045204,4.299620,0.037454,...,3.531484,1.879986,2.273343,2.176359,2.268394,2.254124,13.220630,13.220630,2.211553,3.193130
3,2.163915,1.915785,4.830879,0.699582,1.110310,0.812846,5.856873,1.314613,2.308325,1.252654,...,3.649382,1.827052,2.385817,2.223181,2.313300,2.361125,13.327630,13.327630,2.164692,3.090736
4,0.053629,2.127427,3.664010,0.911225,1.321953,3.560542,6.068515,-0.795673,0.198039,-1.705629,...,3.861024,2.038694,2.597460,2.434824,2.524942,2.572767,13.539273,13.539273,2.376335,3.302378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17620,0.284636,2.194480,4.703390,0.814588,1.239996,1.451831,7.643768,-0.498119,3.479699,-1.214270,...,3.606118,2.043599,2.325969,2.347134,2.320139,2.292332,13.258838,13.258838,2.259281,3.443482
17621,0.459102,2.421001,4.873527,1.102422,1.471217,1.771189,3.612937,-0.324706,1.182912,-1.272753,...,3.855399,2.293863,2.546157,2.490664,2.572189,2.545829,13.512334,13.512334,2.537966,3.876017
17622,2.354685,2.152569,3.585112,0.904770,1.340790,1.417062,7.595169,-0.405086,0.796029,-1.078900,...,3.627826,2.068357,2.344308,2.365683,2.353494,2.348888,13.315394,13.315394,2.232305,3.625496
17623,0.352523,2.472335,4.215219,1.224536,1.660556,1.736829,7.021851,-0.085319,1.115795,-0.759133,...,3.947592,2.388124,2.664075,2.685449,2.673261,2.668655,13.635160,13.635160,2.552071,3.945263


In [4]:
# Define a list of invalid names: no information or virus are invalid
invalid_names = ['f__.g__.s__','g__.s__','virus']

for col in abundance_imputed.columns:
    if any(invalid_name in col for invalid_name in invalid_names):
        abundance_imputed.drop(col, axis=1, inplace=True)

# Extract family, genera, and species from existing column names
def extract_taxonomy(column):
    return '; '.join([t for t in column.split('.') if t.startswith('f__') or t.startswith('g__') or t.startswith('s__')])

new_columns = [extract_taxonomy(column) for column in abundance_imputed.columns]

# Rename columns
abundance_imputed.columns = new_columns

# Add a column containing original patient id for later "merge" step
pid = abundance.index.astype('str')
abundance_imputed.insert(0,'sample_id1',pid)

abundance_imputed

,sample_id1,f__Halomonadaceae; g__Cobetia,f__Thermoanaerobacteraceae; g__Ammonifex,f__Cryomorphaceae; g__Owenweeksia,f__Haloplasmataceae; g__Haloplasma,f__Desulfobacteraceae; g__Desulfosarcina,f__Enterobacteriaceae; g__Siccibacter,f__Acetobacteraceae; g__Asaia,f__Rhodobiaceae; g__Parvibaculum,g__Neosynechococcus,...,f__Clostridiales_Family_XIII; g__Casaltella,f__Chromobacteriaceae; g__Deefgea,f__Phyllobacteriaceae; g__Aquamicrobium,f__Deferribacteraceae; g__Geovibrio,f__Ruminococcaceae; g__Subdoligranulum,,,,,
0,s17489,0.750758,1.698080,1.211822,2.098064,1.901532,0.746481,-0.731171,0.069450,1.183112,...,2.952431,1.053012,1.641701,1.597915,1.615110,1.613142,12.579647,12.579647,1.579534,2.482244
1,s17512,2.957832,2.437834,1.979203,0.569559,1.184304,1.657705,0.251591,2.690162,2.091016,...,3.623247,2.048703,2.357091,2.339657,2.378702,2.338944,13.305450,13.305450,2.332050,2.855626
2,s17498,2.865233,2.223742,1.849716,0.408622,2.003357,1.461779,1.840733,0.527662,1.917301,...,3.531484,1.879986,2.273343,2.176359,2.268394,2.254124,13.220630,13.220630,2.211553,3.193130
3,s17528,1.331853,2.394651,1.813642,0.501072,0.985425,3.488439,4.899925,2.730293,1.865934,...,3.649382,1.827052,2.385817,2.223181,2.313300,2.361125,13.327630,13.327630,2.164692,3.090736
4,s17535,-0.041467,2.606293,2.025285,2.297677,-0.041719,1.378154,2.874528,0.620008,2.077577,...,3.861024,2.038694,2.597460,2.434824,2.524942,2.572767,13.539273,13.539273,2.376335,3.302378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17620,s23,1.492351,2.495612,2.118032,0.773989,0.951887,2.135941,3.213407,1.754071,2.045186,...,3.606118,2.043599,2.325969,2.347134,2.320139,2.292332,13.258838,13.258838,2.259281,3.443482
17621,s20,1.642807,2.709255,2.253808,1.035752,2.675408,2.310780,1.919476,1.929309,2.437095,...,3.855399,2.293863,2.546157,2.490664,2.572189,2.545829,13.512334,13.512334,2.537966,3.876017
17622,s27,1.422109,2.568446,2.202434,0.955879,1.782572,2.043960,1.740752,1.762615,2.213836,...,3.627826,2.068357,2.344308,2.365683,2.353494,2.348888,13.315394,13.315394,2.232305,3.625496
17623,s26,1.741875,2.888213,2.522201,1.275646,1.571824,2.363727,2.060519,2.082382,2.533603,...,3.947592,2.388124,2.664075,2.685449,2.673261,2.668655,13.635160,13.635160,2.552071,3.945263


In [5]:
metadata = pd.read_csv('Metadata-TCGA-All-18116-Samples.csv', index_col=0)
metadata = metadata[metadata['investigation'] == 'TCGA-BLCA'] # Extract only bladder cancer
metadata = pd.DataFrame(metadata)
pid_meta = metadata.index.astype('str')
metadata.insert(0,'sample_id1',pid_meta)
metadata.head

<bound method NDFrame.head of       sample_id1                         gdc_file_uuid  \
s2803      s2803  D20AB198-798F-4512-8498-9FC303E3A4CD   
s2818      s2818  89F55771-696E-444E-8642-20B424F75945   
s2739      s2739  4B54F30A-EAE2-443C-8C0C-F198B72091D1   
s2683      s2683  62E960FB-AAF6-4D7E-B15D-1B5136F6FA37   
s2740      s2740  4E9101B0-24AA-443E-AABD-2A8D28D2F118   
...          ...                                   ...   
s3399      s3399  8C5D03AF-DB90-44D4-BCF7-2EA1DD0FDDE6   
s3397      s3397  F1C7A635-35B5-4838-92FE-AB7E0E4637E4   
s3402      s3402  FBD6B527-9427-4391-A27E-D92B01B74FA8   
s3398      s3398  6A15D446-3828-4624-BB18-D40124C928CF   
s3396      s3396  57523AF1-F9B2-4F88-ABEC-4CF325040361   

                                                filename  age_at_diagnosis  \
s2803  UNCID_2315815.842484b0-6661-4dc2-bf70-b1216ab0...              60.0   
s2818  UNCID_2504338.b76b37cf-c9af-419d-a7c4-f97747de...              73.0   
s2739  TCGA-GV-A3JZ-10A-01D-A21C_120830

In [6]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE) # In case there are invalid string in feature names

SG_CRC_BA = pd.merge(abundance_imputed, metadata, on='sample_id1', how='inner')
SG_CRC_BA.columns = [regex.sub("_",col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in SG_CRC_BA.columns.values]
SG_CRC_BA
#SG_CRC.to_csv('merged_CRC', index = False)


,sample_id1,f__Halomonadaceae; g__Cobetia,f__Thermoanaerobacteraceae; g__Ammonifex,f__Cryomorphaceae; g__Owenweeksia,f__Haloplasmataceae; g__Haloplasma,f__Desulfobacteraceae; g__Desulfosarcina,f__Enterobacteriaceae; g__Siccibacter,f__Acetobacteraceae; g__Asaia,f__Rhodobiaceae; g__Parvibaculum,g__Neosynechococcus,...,portion_weight,aliquot_concentration,analyte_A260A280Ratio,analyte_amount,analyte_type_label,radiation_therapy_code_label,radiation_therapy_site_label,radiation_therapy_type_label,year_of_diagnosis,vital_status_label
0,s2803,1.920406,2.672018,2.367002,0.940031,1.232461,2.305885,2.002862,1.850657,2.367745,...,30.0,0.17,1.90,30.43,RNA,NaN,NaN,NaN,2013.0,Alive
1,s2818,0.575837,2.554444,2.291717,2.598751,1.194319,2.087722,2.225309,0.947783,2.423590,...,30.0,0.17,1.88,57.79,RNA,NaN,NaN,NaN,2012.0,Dead
2,s2739,2.371029,3.318483,2.886261,2.828579,1.511368,2.960159,2.651312,2.581111,2.998545,...,2.0,0.17,NaN,40.03,DNA,NaN,NaN,NaN,2011.0,Alive
3,s2683,-1.297436,3.088465,2.418900,-0.167309,1.477646,2.754875,2.265682,1.883929,2.571443,...,20.0,0.08,NaN,55.07,DNA,8ABD32E6-50C9-44EB-BA59-4572AA05C26C,Primary Tumor Field,External,2011.0,Alive
4,s2740,2.101283,3.048737,2.616515,1.821867,1.013353,2.690412,2.381566,2.311365,2.728799,...,2.0,0.15,NaN,37.23,DNA,NaN,NaN,NaN,2011.0,Alive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,s3399,1.629153,2.532004,2.086196,0.768015,1.767636,2.065759,1.810276,3.350909,2.171045,...,30.0,0.18,1.86,86.39,RNA,NaN,NaN,NaN,2013.0,Alive
725,s3397,1.659999,2.644176,2.185795,0.942269,1.843089,2.282400,1.726946,1.860152,2.282646,...,30.0,0.17,1.83,21.29,RNA,NaN,NaN,NaN,2011.0,Alive
726,s3402,2.391737,3.294074,2.869479,1.585118,0.715287,2.944666,2.446944,2.530973,3.036099,...,30.0,0.17,1.89,42.53,RNA,NaN,NaN,NaN,2007.0,Dead
727,s3398,1.903028,2.654640,2.349624,0.922654,1.867160,2.288508,1.985484,1.833279,2.350368,...,20.0,0.15,2.07,49.65,RNA,NaN,NaN,NaN,2012.0,Dead


In [7]:
class_counts = SG_CRC_BA['sample_type'].value_counts()
print(class_counts)

sample_type
Primary Tumor           557
Blood Derived Normal    124
Solid Tissue Normal      48
Name: count, dtype: int64


In [8]:
# Export final pre-processed meta-transcriptomics data
SG_CRC_BA.to_csv('Poore_BLCA', index=False)